In [1]:
import pandas as pd
import numpy as np
pd.set_option('max_columns',999)
pd.set_option('max_rows',200)

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
import seaborn as sns
sns.set() 
%matplotlib inline 
%config InlineBackend.figure_format = 'retina' 
sns.set_style(style='white')
sns.set_context(context ='paper')

import datetime
import os
from functools import reduce

In [2]:
import selenium as se
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import re

In [3]:
driver = wd.Chrome(r'C:\Users\jeje9\Downloads//chromedriver.exe')

In [40]:
# 크롤링 결과 저장할 df
blurb_df = pd.DataFrame(columns = ['name', 'blurb', 'state', 'category', 'backers', 'funding_rate', 'pledged', 'goal', 'launched', 'deadline'])

### 성공한 프로젝트 크롤링

1. day1 category
2. active category

In [41]:
def webcrawl_for_success(start, end):
    global blurb_df
    print(">>> for DAY1 category")
    for page in range(start,end+1):
        print("==================================")
        print("Collecting page",page,"......")
        print("==================================")

        # load page
        driver.get('https://www.kicktraq.com/dayones/?page={}'.format(page))

        # get list of all project list and info
        prj_list = driver.find_elements_by_xpath("//div[@class='project-infobox']")
        prj_length = len(prj_list)

        for i in range(prj_length):
            x = prj_list[i]

            ## TITLE
            name = x.find_element_by_xpath("h2").text

            ## BLURB
            blurb = x.find_element_by_xpath("div[not(@class)]").text
            
            if blurb != "": ## blurb가 비어있지 않은 애들만 가져옴

                ## CATEGORY
                cat =  x.find_element_by_xpath("div[@class='project-cat']")
                cat_lst = cat.find_elements_by_tag_name("a")
                category = cat_lst[0].text

                ## PROJECT INFORMATION
                info = x.find_element_by_xpath("div[@class='project-infobits']/div[@class='project-details']")
                prj_info = info.text
                # finding rate
                rate_info = info.find_element_by_tag_name("span").text
                percent = int(re.findall('\d+', rate_info)[0])

                if percent >= 100:
                    backers = int(prj_info.split("\n")[0].split(": ")[1])
                    pledged = prj_info.split('\n')[1].split(': ')[1].split(' of ')[0]
                    goal = prj_info.split('\n')[1].split(': ')[1].split(' of ')[1].split(' (')[0]
                    year = prj_info.split('\n')[3].split('(')[1][:-1]
                    launched = prj_info.split('\n')[3].split(": ")[1].split(' -> ')[0]
                    deadline = prj_info.split('\n')[3].split(": ")[1].split(' -> ')[1].split(' (')[0]
                    row = [name, blurb, 'success', category, backers, percent, pledged, goal, year + ' ' + launched, year + ' ' + deadline]
                    blurb_df = blurb_df.append(pd.Series(row, index=['name', 'blurb', 'state', 'category', 'backers', 'funding_rate', 'pledged', 'goal', 'launched', 'deadline']), ignore_index=True)
                
       
    print(">>> for ACTIVE category")
    for page in range(start,end+1):
        print("==================================")
        print("Collecting page",page,"......")
        print("==================================")
        
        # load page
        driver.get('https://www.kicktraq.com/projects/?page={}'.format(page))

        # get list of all project list and info
        prj_list = driver.find_elements_by_xpath("//div[@class='project-infobox']")
        prj_length = len(prj_list)

        for i in range(prj_length):
            x = prj_list[i]
            
            ## PROJECT INFORMATION
            info = x.find_element_by_xpath("div[@class='project-infobits']/div[@class='project-details']")
            prj_info = info.text
            
            if (len(prj_info.split("\n")) > 3) and (prj_info.split("\n")[4] == 'time left: 0 days, 0 hours, 0 minutes (closing/closed)'):

                ## TITLE
                name = x.find_element_by_xpath("h2").text
                
                ## BLURB
                blurb = x.find_element_by_xpath("div[not(@class)]").text 
                
                if blurb != "": ## blurb가 비어있지 않은 애들만 가져옴

                    ## CATEGORY
                    cat =  x.find_element_by_xpath("div[@class='project-cat']")
                    cat_lst = cat.find_elements_by_tag_name("a")
                    category = cat_lst[0].text

            
                    # finding rate
                    rate_info = info.find_element_by_tag_name("span").text
                    percent = int(re.findall('\d+', rate_info)[0])

                    if percent >= 100:
                        backers = int(prj_info.split("\n")[0].split(": ")[1])
                        pledged = prj_info.split('\n')[1].split(': ')[1].split(' of ')[0]
                        goal = prj_info.split('\n')[1].split(': ')[1].split(' of ')[1].split(' (')[0]
                        year = prj_info.split('\n')[3].split('(')[1][:-1]
                        launched = prj_info.split('\n')[3].split(": ")[1].split(' -> ')[0]
                        deadline = prj_info.split('\n')[3].split(": ")[1].split(' -> ')[1].split(' (')[0]
                        row = [name, blurb, 'success', category, backers, percent, pledged, goal, year + ' ' + launched, year + ' ' + deadline]
                        blurb_df = blurb_df.append(pd.Series(row, index=['name', 'blurb', 'state', 'category', 'backers', 'funding_rate', 'pledged', 'goal', 'launched', 'deadline']), ignore_index=True)

In [42]:
webcrawl_for_success(1,3)

>>> for DAY1 category
>>> for ACTIVE category


### 실패한 프로젝트 크롤링

1. archived category
2. active category

In [43]:
def webcrawl_for_fail(start, end):
    global blurb_df
    print(">>> for ARCHIVED category")
    for page in range(start,end+1):
        print("==================================")
        print("Collecting page",page,"......")
        print("==================================")

        # load page
        driver.get('https://www.kicktraq.com/archive/?page={}'.format(page))

        # get list of all project list and info
        prj_list = driver.find_elements_by_xpath("//div[@class='project-infobox']")
        prj_length = len(prj_list)

        for i in range(prj_length):
            x = prj_list[i]

            ## TITLE
            name = x.find_element_by_xpath("h2").text

            ## BLURB
            blurb = x.find_element_by_xpath("div[not(@class)]").text
            
            if blurb != "": ## blurb가 비어있지 않은 애들만 가져옴

                ## CATEGORY
                cat =  x.find_element_by_xpath("div[@class='project-cat']")
                cat_lst = cat.find_elements_by_tag_name("a")
                category = cat_lst[0].text

                ## PROJECT INFORMATION
                info = x.find_element_by_xpath("div[@class='project-infobits']/div[@class='project-details']")
                prj_info = info.text
                # finding rate
                rate_info = info.find_element_by_tag_name("span").text
                percent = int(re.findall('\d+', rate_info)[0])

                if percent < 100:
                    backers = int(prj_info.split("\n")[0].split(": ")[1])
                    pledged = prj_info.split('\n')[1].split(': ')[1].split(' of ')[0]
                    goal = prj_info.split('\n')[1].split(': ')[1].split(' of ')[1].split(' (')[0]
                    year = prj_info.split('\n')[2].split('(')[1][:-1]
                    launched = prj_info.split('\n')[2].split(": ")[1].split(' -> ')[0]
                    deadline = prj_info.split('\n')[2].split(": ")[1].split(' -> ')[1].split(' (')[0]
                    row = [name, blurb, 'failed', category, backers, percent, pledged, goal, year + ' ' + launched, year + ' ' + deadline]
                    blurb_df = blurb_df.append(pd.Series(row, index=['name', 'blurb', 'state', 'category', 'backers', 'funding_rate', 'pledged', 'goal', 'launched', 'deadline']), ignore_index=True)
                
    print(">>> for ACTIVE category")
    for page in range(start,end+1):
        print("==================================")
        print("Collecting page",page,"......")
        print("==================================")
        
        # load page
        driver.get('https://www.kicktraq.com/projects/?page={}'.format(page))

        # get list of all project list and info
        prj_list = driver.find_elements_by_xpath("//div[@class='project-infobox']")
        prj_length = len(prj_list)

        for i in range(prj_length):
            x = prj_list[i]
            
            ## PROJECT INFORMATION
            info = x.find_element_by_xpath("div[@class='project-infobits']/div[@class='project-details']")
            prj_info = info.text
            
            if len(prj_info.split("\n")) == 5:

                ## TITLE
                name = x.find_element_by_xpath("h2").text

                ## BLURB
                blurb = x.find_element_by_xpath("div[not(@class)]").text 
                
                if blurb != "": ## blurb가 비어있지 않은 애들만 가져옴

                    ## CATEGORY
                    cat =  x.find_element_by_xpath("div[@class='project-cat']")
                    cat_lst = cat.find_elements_by_tag_name("a")
                    category = cat_lst[0].text

            
                    # finding rate
                    rate_info = info.find_element_by_tag_name("span").text
                    percent = int(re.findall('\d+', rate_info)[0])

                    if percent < 100:
                        backers = int(prj_info.split("\n")[0].split(": ")[1])
                        pledged = prj_info.split('\n')[1].split(': ')[1].split(' of ')[0]
                        goal = prj_info.split('\n')[1].split(': ')[1].split(' of ')[1].split(' (')[0]
                        year = prj_info.split('\n')[3].split('(')[1][:-1]
                        launched = prj_info.split('\n')[3].split(": ")[1].split(' -> ')[0]
                        deadline = prj_info.split('\n')[3].split(": ")[1].split(' -> ')[1].split(' (')[0]
                        row = [name, blurb, 'failed', category, backers, percent, pledged, goal, year + ' ' + launched, year + ' ' + deadline]
                        blurb_df = blurb_df.append(pd.Series(row, index=['name', 'blurb', 'state', 'category', 'backers', 'funding_rate', 'pledged', 'goal', 'launched', 'deadline']), ignore_index=True)

In [44]:
webcrawl_for_fail(1,3)

>>> for ARCHIVED category
>>> for ACTIVE category


In [46]:
blurb_df.head()

,name,blurb,state,category,backers,funding_rate,pledged,goal,launched,deadline
0,Vintage Reborn: New Vintage Chicken Nugget Pla...,Remastering Jerry's Nugget's rare predecessor ...,success,Games,519,348,"$34,880","$10,000",2019 October 8th,2019 November 15th
1,Intégrale des sonates de Bach pour clavecin et...,Les 6 sonates de Bach pour clavecin obligé et ...,success,Art,52,100,"€7,500","€7,500",2019 September 22nd,2019 November 21st
2,Sterkmann Travel Bag | The Most Organized Bag ...,19 different pockets and compartments guarante...,success,Design,1228,1331,"$199,671","$15,000",2019 October 2nd,2019 November 21st
3,Trexad Mundo | The Travel Pack Created by the ...,"We asked 2,701 people what they wanted in thei...",success,Design,743,657,"$98,558","$15,000",2019 October 2nd,2019 November 21st
4,"MolaBox: The Reusable, Collapsible Lunchbox Fr...",A super f**king sexy lunchbox | Designed to el...,success,Design,255,246,"$24,605","$10,000",2019 October 22nd,2019 November 21st


### 기존 데이터 모양으로 전처리 (날짜, 컬럼이름 맞춰줌)

In [27]:
projects = pd.read_csv('projects_fin.csv')

In [28]:
print(projects.shape)
projects.head()

(372572, 23)


,Unnamed: 0,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,state_new,deadline_dt,launched_dt,term,term_str,term_bin,usd_goal_real_bin
0,0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95,failed,2015-10-09,2015-08-11,59,59,6,3
1,1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00,failed,2017-11-01,2017-09-02,60,60,6,6
2,2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00,failed,2013-02-26,2013-01-12,45,45,5,6
3,3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00,failed,2012-04-16,2012-03-17,30,30,4,4
4,4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00,failed,2015-08-29,2015-07-04,56,56,6,6


In [12]:
blurb_df.columns = ['name', 'blurb', 'state_new', 'main_category', 'backers', 'funding_rate', 'pledged', 'goal',
       'launched', 'deadline']

In [47]:
blurb_df.head()

,name,blurb,state,category,backers,funding_rate,pledged,goal,launched,deadline
0,Vintage Reborn: New Vintage Chicken Nugget Pla...,Remastering Jerry's Nugget's rare predecessor ...,success,Games,519,348,"$34,880","$10,000",2019 October 8th,2019 November 15th
1,Intégrale des sonates de Bach pour clavecin et...,Les 6 sonates de Bach pour clavecin obligé et ...,success,Art,52,100,"€7,500","€7,500",2019 September 22nd,2019 November 21st
2,Sterkmann Travel Bag | The Most Organized Bag ...,19 different pockets and compartments guarante...,success,Design,1228,1331,"$199,671","$15,000",2019 October 2nd,2019 November 21st
3,Trexad Mundo | The Travel Pack Created by the ...,"We asked 2,701 people what they wanted in thei...",success,Design,743,657,"$98,558","$15,000",2019 October 2nd,2019 November 21st
4,"MolaBox: The Reusable, Collapsible Lunchbox Fr...",A super f**king sexy lunchbox | Designed to el...,success,Design,255,246,"$24,605","$10,000",2019 October 22nd,2019 November 21st


In [48]:
blurb_df['start_year'] = blurb_df.apply(lambda x: x['launched'].split(" ")[0], axis = 1)
blurb_df['start_month'] = blurb_df.apply(lambda x: x['launched'].split(" ")[1], axis = 1)
blurb_df['start_day'] = blurb_df.apply(lambda x: x['launched'].split(" ")[2], axis = 1)

blurb_df['end_year'] = blurb_df.apply(lambda x: x['deadline'].split(" ")[0], axis = 1)
blurb_df['end_month'] = blurb_df.apply(lambda x: x['deadline'].split(" ")[1], axis = 1)
blurb_df['end_day'] = blurb_df.apply(lambda x: x['deadline'].split(" ")[2], axis = 1)

In [49]:
blurb_df['start_month_mod'] = blurb_df.apply(lambda x: '01' if x['start_month'] == 'January'
                                             else '02' if x['start_month'] == 'Feburary'
                                             else '03' if x['start_month'] == 'March'
                                             else '04' if x['start_month'] == 'April'
                                             else '05' if x['start_month'] == 'May'
                                             else '06' if x['start_month'] == 'June'
                                             else '07' if x['start_month'] == 'July'
                                             else '08' if x['start_month'] == 'August'
                                             else '09' if x['start_month'] == 'September'
                                             else '10' if x['start_month'] == 'October'
                                             else '11' if x['start_month'] == 'November'
                                             else '12' , axis = 1)

blurb_df['end_month_mod'] = blurb_df.apply(lambda x: '01' if x['end_month'] == 'January'
                                           else '02' if x['end_month'] == 'Feburary'
                                           else '03' if x['end_month'] == 'March'
                                           else '04' if x['end_month'] == 'April'
                                           else '05' if x['end_month'] == 'May'
                                           else '06' if x['end_month'] == 'June'
                                           else '07' if x['end_month'] == 'July'
                                           else '08' if x['end_month'] == 'August'
                                           else '09' if x['end_month'] == 'September'
                                           else '10' if x['end_month'] == 'October'
                                           else '11' if x['end_month'] == 'November'
                                           else '12' , axis = 1)

In [50]:
blurb_df['start_day_mod'] = blurb_df.apply(lambda x: x['start_day'][:-2], axis = 1)

blurb_df['end_day_mod'] = blurb_df.apply(lambda x: x['end_day'][:-2], axis = 1)

In [51]:
blurb_df['launched_new'] = blurb_df.apply(lambda x: x['start_year'] + '-' + x['start_month_mod'] + '-' + x['start_day_mod']
                                          , axis = 1)

blurb_df['deadline_new'] = blurb_df.apply(lambda x: x['end_year'] + '-' + x['end_month_mod'] + '-' + x['end_day_mod']
                                          , axis = 1)

In [52]:
blurb_df.columns

Index(['name', 'blurb', 'state', 'category', 'backers', 'funding_rate',
       'pledged', 'goal', 'launched', 'deadline', 'start_year', 'start_month',
       'start_day', 'end_year', 'end_month', 'end_day', 'start_month_mod',
       'end_month_mod', 'start_day_mod', 'end_day_mod', 'launched_new',
       'deadline_new'],
      dtype='object')

In [53]:
blurb_df = blurb_df.drop(['start_year', 'start_month','start_day', 'end_year', 'end_month', 'end_day', 'start_month_mod',
                          'end_month_mod', 'start_day_mod', 'end_day_mod'], axis = 1)

In [54]:
blurb_df['deadline_dt'] = blurb_df.apply(lambda x: datetime.datetime.strptime(x['deadline_new'], "%Y-%m-%d").date(), axis = 1)
blurb_df['launched_dt'] = blurb_df.apply(lambda x: datetime.datetime.strptime(x['launched_new'], "%Y-%m-%d").date(), axis = 1)

In [56]:
blurb_df['term'] = blurb_df.apply(lambda x: int(str(x['deadline_dt'] - x['launched_dt']).split(" ")[0]), axis = 1)
blurb_df['term_str'] = blurb_df.apply(lambda x: str(x['term']), axis = 1)

In [57]:
# 최종 모양
blurb_df.head()

,name,blurb,state,category,backers,funding_rate,pledged,goal,launched,deadline,launched_new,deadline_new,deadline_dt,launched_dt,term,term_str
0,Vintage Reborn: New Vintage Chicken Nugget Pla...,Remastering Jerry's Nugget's rare predecessor ...,success,Games,519,348,"$34,880","$10,000",2019 October 8th,2019 November 15th,2019-10-8,2019-11-15,2019-11-15,2019-10-08,38,38
1,Intégrale des sonates de Bach pour clavecin et...,Les 6 sonates de Bach pour clavecin obligé et ...,success,Art,52,100,"€7,500","€7,500",2019 September 22nd,2019 November 21st,2019-09-22,2019-11-21,2019-11-21,2019-09-22,60,60
2,Sterkmann Travel Bag | The Most Organized Bag ...,19 different pockets and compartments guarante...,success,Design,1228,1331,"$199,671","$15,000",2019 October 2nd,2019 November 21st,2019-10-2,2019-11-21,2019-11-21,2019-10-02,50,50
3,Trexad Mundo | The Travel Pack Created by the ...,"We asked 2,701 people what they wanted in thei...",success,Design,743,657,"$98,558","$15,000",2019 October 2nd,2019 November 21st,2019-10-2,2019-11-21,2019-11-21,2019-10-02,50,50
4,"MolaBox: The Reusable, Collapsible Lunchbox Fr...",A super f**king sexy lunchbox | Designed to el...,success,Design,255,246,"$24,605","$10,000",2019 October 22nd,2019 November 21st,2019-10-22,2019-11-21,2019-11-21,2019-10-22,30,30
